In [95]:
import numpy as np 
import pandas as pd # 
from matplotlib import pyplot as plt
import seaborn as sns

from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from components.data_pipeline import dataPipeline

import warnings
warnings.filterwarnings('ignore')

In [79]:
#TESTES
df_test = pd.read_csv('input/teste1.csv')
df_test, _ = dataPipeline(df_test)
df_test = df_test.sort_values(by='data_semana_comercial')


df_test2 = pd.read_csv('input/teste2.csv')
df_test2, _ = dataPipeline(df_test2)
df_test2 = df_test2.sort_values(by='data_semana_comercial')

Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado
Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado


In [80]:
#TREINO
df = pd.read_csv('input/treino.csv')
df, _ = dataPipeline(df)
df = df.sort_values(by='data_semana_comercial')

Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado


# Prophet

In [81]:
df_prophet = df.copy()
df_prophet.data_semana_comercial = pd.to_datetime(df_prophet.data_semana_comercial)
df_prophet.drop('cod_loja', axis=1, inplace=True)
df_prophet.columns = ['ds', 'y']
df_prophet.head()

,ds,y
60502,2017-01-01,18
32686,2017-01-08,3
7667,2017-01-15,16
38204,2017-01-22,7
76966,2017-01-29,22


In [82]:
model = Prophet()
model.fit(df_prophet)

17:11:18 - cmdstanpy - INFO - Chain [1] start processing
17:11:18 - cmdstanpy - INFO - Chain [1] done processing


In [83]:
df_prophet_test = df_test.copy()
df_prophet_test.drop('cod_loja', axis=1, inplace=True)
df_prophet_test.data_semana_comercial = pd.to_datetime(df_prophet_test.data_semana_comercial)
df_prophet_test.columns = ['ds', 'venda']

df_prophet_test2 = df_test2.copy()
df_prophet_test2.drop('cod_loja', axis=1, inplace=True)
df_prophet_test2.data_semana_comercial = pd.to_datetime(df_prophet_test2.data_semana_comercial)
df_prophet_test2.columns = ['ds', 'venda']

In [100]:
df_cv = cross_validation(model, initial='30 days', period='7 days', horizon = '')
df_cv.head()

Seasonality has period of 365.25 days which is larger than initial window. Consider increasing initial.
  0%|          | 0/289 [00:00<?, ?it/s]17:22:18 - cmdstanpy - INFO - Chain [1] start processing
17:22:18 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 1/289 [00:00<01:18,  3.66it/s]17:22:18 - cmdstanpy - INFO - Chain [1] start processing
17:22:18 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 2/289 [00:00<01:08,  4.20it/s]17:22:18 - cmdstanpy - INFO - Chain [1] start processing
17:22:19 - cmdstanpy - INFO - Chain [1] done processing
  1%|          | 3/289 [00:00<01:00,  4.76it/s]17:22:19 - cmdstanpy - INFO - Chain [1] start processing
17:22:19 - cmdstanpy - INFO - Chain [1] done processing
  1%|▏         | 4/289 [00:00<00:55,  5.12it/s]17:22:19 - cmdstanpy - INFO - Chain [1] start processing
17:22:19 - cmdstanpy - INFO - Chain [1] done processing
  2%|▏         | 5/289 [00:00<00:50,  5.59it/s]17:22:19 - cmdstanpy - INFO - Chain [1] start processing


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,2017-02-12,-155.685348,-155.685348,-155.685347,45,2017-02-05
1,2017-02-19,330.225366,330.225366,330.225366,38,2017-02-12
2,2017-02-26,-49.980095,-53.677062,-46.318958,45,2017-02-19
3,2017-03-05,88.502202,83.916548,92.515248,49,2017-02-26
4,2017-03-12,3.391511,-0.731036,7.389154,61,2017-03-05


In [101]:
df_p = performance_metrics(df_cv)
df_p

,horizon,mse,rmse,mae,mdape,smape,coverage
0,7 days,574094.414343,757.690184,473.788027,0.74599,0.93558,0.6225


In [92]:
forecast1.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
11,2022-06-05,108.656623,-259.630299,1433.681365,108.656623,108.656623,465.091482,465.091482,465.091482,465.091482,465.091482,465.091482,0.0,0.0,0.0,573.748105
12,2022-06-12,99.059358,-472.333609,1353.374142,99.059358,99.059358,356.030545,356.030545,356.030545,356.030545,356.030545,356.030545,0.0,0.0,0.0,455.089903
13,2022-06-12,99.059358,-403.851169,1346.431497,99.059358,99.059358,356.030545,356.030545,356.030545,356.030545,356.030545,356.030545,0.0,0.0,0.0,455.089903
14,2022-06-19,89.462093,-390.407921,1311.871843,89.462093,89.462093,372.024485,372.024485,372.024485,372.024485,372.024485,372.024485,0.0,0.0,0.0,461.486578
15,2022-06-19,89.462093,-462.888742,1348.096453,89.462093,89.462093,372.024485,372.024485,372.024485,372.024485,372.024485,372.024485,0.0,0.0,0.0,461.486578


In [88]:
df_

In [90]:
df_prophet_test.head()

,ds,venda,yhat
823,2022-05-01,572,NaN
1430,2022-05-01,150,NaN
707,2022-05-08,605,NaN
2222,2022-05-08,202,NaN
62,2022-05-15,1269,NaN


In [ ]:
df_406 = df[df['cod_loja'] == 406]
df_407 = df[df['cod_loja'] == 407]

In [ ]:
df_406['next_week'] = df_406['venda'].shift(-1)
df_406['next_2week'] = df_406['venda'].shift(-2)